In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader, Dataset
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [2]:
train_data = pd.read_csv("./arxiv_train.csv")
train_data

,Unnamed: 0,abstract,label
0,31716,Automatic meeting analysis is an essential f...,eess
1,89533,We propose a protocol to encode classical bi...,quant-ph
2,82700,A number of physically intuitive results for...,quant-ph
3,78830,In the last decade rare-earth hexaborides ha...,physics
4,94948,We introduce the weak barycenter of a family...,stat
...,...,...,...
79995,27913,"In this paper, the sum secure degrees of fre...",cs
79996,94441,"In areas of application, including actuarial...",stat
79997,33015,Failure detection is employed in the industr...,eess
79998,942,As part of the ongoing effort to characteriz...,astro-ph


In [3]:
# Load the train set
test_data = pd.read_csv("./arxiv_test.csv")
test_data

,Unnamed: 0,abstract,label
0,64481,We describe a shape derivative approach to p...,math
1,48104,We study displaced signatures of sneutrino p...,hep-ph
2,48233,High precision studies of Beyond-Standard-Mo...,hep-ph
3,49026,We find that a class of models of MeV-GeV da...,hep-ph
4,37957,Knowledge of power grid's topology during ca...,eess
...,...,...,...
19995,50391,We explore the dynamics of a simple class of...,hep-th
19996,63534,In this paper one construction of compositio...,math
19997,16712,The Random-First-Order-Transition theory of ...,cond-mat
19998,6596,Accurate chemical abundance measurements of ...,astro-ph


In [6]:
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters and numbers
    tokens = word_tokenize(text.lower())  # Tokenize the text
    stop_words = set(stopwords.words("english"))  # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()  # Lemmatize each word
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    stemmer = PorterStemmer()  # Stem each word
    tokens = [stemmer.stem(word) for word in tokens]
    clean_text = " ".join(tokens)  # Join tokens back into a single string
    return clean_text


train_data["clean_abstract"] = train_data["abstract"].apply(clean_text)
test_data["clean_abstract"] = test_data["abstract"].apply(clean_text)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\SaharKadkhodaMasoumA/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\SaharKadkhodaMasoumA\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [4]:
count_vectorizer = CountVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [5]:
X_train_count = count_vectorizer.fit_transform(train_data["abstract"]).toarray()
X_test_count = count_vectorizer.transform(test_data["abstract"]).toarray()

In [6]:
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data["abstract"]).toarray()
X_test_tfidf = tfidf_vectorizer.transform(test_data["abstract"]).toarray()

In [7]:
label_encoder = {label: idx for idx, label in enumerate(train_data["label"].unique())}
train_data["label_encoded"] = train_data["label"].map(label_encoder)
test_data["label_encoded"] = test_data["label"].map(label_encoder)

In [8]:
y_train = torch.tensor(train_data["label_encoded"].values)
y_test = torch.tensor(test_data["label_encoded"].values)

In [9]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [10]:
#class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(FFNN, self).__init__()
        self.num_layers = num_layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 1)]
        )
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.fc2(x)
        return x

In [17]:
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, activation=nn.ReLU()):
        super(FFNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_layers = nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.hidden_layers.append(nn.Linear(input_size, hidden_size))
            elif i == num_layers - 1:
                self.hidden_layers.append(nn.Linear(hidden_size, output_size))
            else:
                self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
            self.hidden_layers.append(activation)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        return x



In [11]:
# Function to train the model
def train_model(model, criterion, optimizer, train_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss}")

In [12]:
# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.float())
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

In [13]:
# Define train and test datasets
train_dataset_count = CustomDataset(X_train_count, y_train)
test_dataset_count = CustomDataset(X_test_count, y_test)

train_dataset_tfidf = CustomDataset(X_train_tfidf, y_train)
test_dataset_tfidf = CustomDataset(X_test_tfidf, y_test)

# Define data loaders
train_loader_count = DataLoader(train_dataset_count, batch_size=64, shuffle=True)
test_loader_count = DataLoader(test_dataset_count, batch_size=64, shuffle=False)

train_loader_tfidf = DataLoader(train_dataset_tfidf, batch_size=64, shuffle=True)
test_loader_tfidf = DataLoader(test_dataset_tfidf, batch_size=64, shuffle=False)

In [14]:
# Train and evaluate FFNN with CountVectorizer and different numbers of layers
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
for num_layers in [1, 2, 3]:
    print(f"Training FFNN with CountVectorizer and {num_layers} layers")
    ffnn_count = FFNN(input_size_count, 256, output_size_count, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
    train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_count, test_loader_count)
    print()

Training FFNN with CountVectorizer and 1 layers
Epoch [1/10], Loss: 743.0623899400234
Epoch [2/10], Loss: 445.8628660440445
Epoch [3/10], Loss: 263.4596190750599
Epoch [4/10], Loss: 114.98958802036941
Epoch [5/10], Loss: 38.809479392599314
Epoch [6/10], Loss: 14.333205292350613
Epoch [7/10], Loss: 7.945223589427769
Epoch [8/10], Loss: 20.164672709885053
Epoch [9/10], Loss: 16.07088966111769
Epoch [10/10], Loss: 7.142239848835743
Evaluation on Test Set:
Accuracy: 0.8107, Precision: 0.8109409515555879, Recall: 0.8104510494535951, F1-Score: 0.8096136114078154

Training FFNN with CountVectorizer and 2 layers
Epoch [1/10], Loss: 749.5075382143259
Epoch [2/10], Loss: 440.41998664289713
Epoch [3/10], Loss: 258.40501895174384
Epoch [4/10], Loss: 110.69346080627292
Epoch [5/10], Loss: 52.09413174400106
Epoch [6/10], Loss: 48.67403827956878
Epoch [7/10], Loss: 33.54433995686122
Epoch [8/10], Loss: 30.207818542796304
Epoch [9/10], Loss: 32.536002864377224
Epoch [10/10], Loss: 25.806746830567135
E

In [18]:
# Train and evaluate FFNN with CountVectorizer and different activation function
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
activation_functions = [nn.LeakyReLU(), nn.ELU(), nn.Sigmoid(), nn.Softmax(dim=1)]

for activation_function in activation_functions:
    print(f"Training FFNN with CountVectorizer and {activation_function.__class__.__name__} as an activation function")
    ffnn_count = FFNN(input_size_count, 256, output_size_count, num_layers=2, activation=activation_function)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
    train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_count, test_loader_count)
    print()


Training FFNN with CountVectorizer and LeakyReLU as an activation function
Epoch [1/10], Loss: 779.1226999759674
Epoch [2/10], Loss: 461.2558950930834
Epoch [3/10], Loss: 290.4565050601959
Epoch [4/10], Loss: 142.1204579025507
Epoch [5/10], Loss: 54.91493967361748
Epoch [6/10], Loss: 22.617049310123548
Epoch [7/10], Loss: 17.038059719197918
Epoch [8/10], Loss: 20.81000182213029
Epoch [9/10], Loss: 12.523910178308142
Epoch [10/10], Loss: 10.43505616929906
Evaluation on Test Set:
Accuracy: 0.8091, Precision: 0.8084935102365746, Recall: 0.8085558081893665, F1-Score: 0.8082112300010884

Training FFNN with CountVectorizer and ELU as an activation function
Epoch [1/10], Loss: 733.471880197525
Epoch [2/10], Loss: 483.33389146625996
Epoch [3/10], Loss: 373.9589847549796
Epoch [4/10], Loss: 272.5039286017418
Epoch [5/10], Loss: 177.92759406752884
Epoch [6/10], Loss: 97.2131339032203
Epoch [7/10], Loss: 48.75164867145941
Epoch [8/10], Loss: 25.524274023831822
Epoch [9/10], Loss: 18.6469395519234

In [21]:
input_size_tfidf = X_train_tfidf.shape[1]
output_size_tfidf = len(label_encoder)
for num_layers in [1, 2, 3]:
    print(f"Training FFNN with TF-IDF and {num_layers} layers")
    ffnn_tfidf = FFNN(input_size_tfidf, 256, output_size_tfidf, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_tfidf.parameters(), lr=0.001)
    train_model(ffnn_tfidf, criterion, optimizer, train_loader_tfidf, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_tfidf, test_loader_tfidf)
    print()

Training FFNN with TF-IDF and 1 layers
Epoch [1/10], Loss: 4564.730442523956
Epoch [2/10], Loss: 2367.812207341194
Epoch [3/10], Loss: 1836.384837269783
Epoch [4/10], Loss: 1624.8815511465073
Epoch [5/10], Loss: 1509.1928326487541
Epoch [6/10], Loss: 1435.4432886242867
Epoch [7/10], Loss: 1383.907634139061
Epoch [8/10], Loss: 1345.6823671460152
Epoch [9/10], Loss: 1315.9424677491188
Epoch [10/10], Loss: 1291.9441948533058
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.74615, Precision: 0.6750732653302313, Recall: 0.7452886692498367, F1-Score: 0.7071678329751698

Training FFNN with TF-IDF and 2 layers
Epoch [1/10], Loss: 2126.0801639556885
Epoch [2/10], Loss: 1998.6933007240295
Epoch [3/10], Loss: 1963.53854906559
Epoch [4/10], Loss: 1933.5398559570312
Epoch [5/10], Loss: 1902.8719158172607
Epoch [6/10], Loss: 1870.8874015808105
Epoch [7/10], Loss: 1841.279080927372
Epoch [8/10], Loss: 1816.0729239583015
Epoch [9/10], Loss: 1797.565864264965
Epoch [10/10], Loss: 1785.2200683951378
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.32985, Precision: 0.23981533609427083, Recall: 0.3295950278342651, F1-Score: 0.2573339728174858

Training FFNN with TF-IDF and 3 layers
Epoch [1/10], Loss: 1472.6598109602928
Epoch [2/10], Loss: 1261.7234600782394
Epoch [3/10], Loss: 1173.2666721642017
Epoch [4/10], Loss: 1074.9902056455612
Epoch [5/10], Loss: 982.0924243330956
Epoch [6/10], Loss: 927.5030905008316
Epoch [7/10], Loss: 910.7739372253418
Epoch [8/10], Loss: 910.7030127942562
Epoch [9/10], Loss: 911.3091934025288
Epoch [10/10], Loss: 909.6576625108719
Evaluation on Test Set:
Accuracy: 0.56975, Precision: 0.508187061069251, Recall: 0.5690386647150071, F1-Score: 0.5193990644210014



C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
input_size_tfidf = X_train_tfidf.shape[1]
output_size_tfidf = len(label_encoder)
activation_functions = [nn.LeakyReLU(), nn.ELU(), nn.Sigmoid(), nn.Softmax(dim=1)]

for activation_function in activation_functions:
    print(f"Training FFNN with tfidf and {activation_function.__class__.__name__} as an activation function")
    ffnn_tfidf = FFNN(input_size_tfidf, 256, output_size_tfidf, num_layers=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_tfidf.parameters(), lr=0.001)
    train_model(ffnn_tfidf, criterion, optimizer, train_loader_tfidf, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_tfidf, test_loader_tfidf)
    print()

Training FFNN with tfidf and LeakyReLU as an activation function
Epoch [1/10], Loss: 1989.0983240008354
Epoch [2/10], Loss: 1353.0579169988632
Epoch [3/10], Loss: 1045.6689096987247
Epoch [4/10], Loss: 918.1834497153759
Epoch [5/10], Loss: 860.5946475565434
Epoch [6/10], Loss: 805.3669632673264
Epoch [7/10], Loss: 753.17565099895
Epoch [8/10], Loss: 705.7292978316545
Epoch [9/10], Loss: 669.6869882196188
Epoch [10/10], Loss: 643.85013666749
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.67525, Precision: 0.5958855356498138, Recall: 0.6769820138908375, F1-Score: 0.622319832409534

Training FFNN with tfidf and ELU as an activation function
Epoch [1/10], Loss: 1741.1844599843025
Epoch [2/10], Loss: 1542.819808781147
Epoch [3/10], Loss: 1493.7148686647415
Epoch [4/10], Loss: 1455.0220013856888
Epoch [5/10], Loss: 1417.0658017992973
Epoch [6/10], Loss: 1376.7458293437958
Epoch [7/10], Loss: 1336.772608935833
Epoch [8/10], Loss: 1299.9106670618057
Epoch [9/10], Loss: 1269.2381512522697
Epoch [10/10], Loss: 1246.8699571490288
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5538, Precision: 0.4752740156301395, Recall: 0.5567484209814866, F1-Score: 0.5018888406457622

Training FFNN with tfidf and Sigmoid as an activation function
Epoch [1/10], Loss: 1697.6441856622696
Epoch [2/10], Loss: 1502.6167083978653
Epoch [3/10], Loss: 1452.2089486718178
Epoch [4/10], Loss: 1411.4019531607628
Epoch [5/10], Loss: 1372.9568030238152
Epoch [6/10], Loss: 1334.5735786557198
Epoch [7/10], Loss: 1298.6759565472603
Epoch [8/10], Loss: 1267.5561074614525
Epoch [9/10], Loss: 1245.2560103535652
Epoch [10/10], Loss: 1229.4111191034317
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.58055, Precision: 0.48890891388077584, Recall: 0.5816266085872539, F1-Score: 0.5176493215945227

Training FFNN with tfidf and Softmax as an activation function
Epoch [1/10], Loss: 1077.9348317086697
Epoch [2/10], Loss: 765.788494348526
Epoch [3/10], Loss: 690.202105641365
Epoch [4/10], Loss: 629.3187815696001
Epoch [5/10], Loss: 568.375945404172
Epoch [6/10], Loss: 508.4011527746916
Epoch [7/10], Loss: 450.0943462923169
Epoch [8/10], Loss: 400.73719269037247
Epoch [9/10], Loss: 363.0493036210537
Epoch [10/10], Loss: 337.2618256136775
Evaluation on Test Set:
Accuracy: 0.7244, Precision: 0.6799357441171696, Recall: 0.724543853929455, F1-Score: 0.696241392087842



C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
# model for evaluating hidden size
import torch.nn as nn

class FFNN_hidden_size(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation=nn.ReLU()):
        super(FFNN_hidden_size, self).__init__()  # Corrected super call
        self.num_layers = len(hidden_sizes) + 1
        self.hidden_layers = nn.ModuleList()
        for i in range(self.num_layers - 1):
            if i == 0:
                self.hidden_layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                self.hidden_layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.hidden_layers.append(activation)
        self.hidden_layers.append(nn.Linear(hidden_sizes[-1], output_size))

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        return x


In [31]:
# for count verctorize
import torch.optim as optim

input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
hidden_sizes = [64, 128, 256]

for hidden_size in hidden_sizes:
    print(f"Training FFNN with CountVectorizer and {hidden_size} as hidden size")
    ffnn_count = FFNN_hidden_size(input_size_count, [hidden_size], output_size_count, activation=nn.Sigmoid())
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
    train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_count, test_loader_count)
    print()


Training FFNN with CountVectorizer and 64 as hidden size
Epoch [1/10], Loss: 941.8634288012981
Epoch [2/10], Loss: 520.6390979588032
Epoch [3/10], Loss: 429.6217685043812
Epoch [4/10], Loss: 363.66021063923836
Epoch [5/10], Loss: 308.9180175587535
Epoch [6/10], Loss: 258.53121416270733
Epoch [7/10], Loss: 214.06433810666203
Epoch [8/10], Loss: 172.53173238411546
Epoch [9/10], Loss: 135.47612247988582
Epoch [10/10], Loss: 102.8923531267792
Evaluation on Test Set:
Accuracy: 0.8097, Precision: 0.8089457577534478, Recall: 0.8093464471654487, F1-Score: 0.8089794936234525

Training FFNN with CountVectorizer and 128 as hidden size
Epoch [1/10], Loss: 851.1252954006195
Epoch [2/10], Loss: 507.1571571826935
Epoch [3/10], Loss: 415.8249719515443
Epoch [4/10], Loss: 347.1114614382386
Epoch [5/10], Loss: 286.2970133051276
Epoch [6/10], Loss: 228.85414877906442
Epoch [7/10], Loss: 175.76912094652653
Epoch [8/10], Loss: 128.94227942824364
Epoch [9/10], Loss: 88.70155765675008
Epoch [10/10], Loss: 58

In [32]:
num_epochs = 10
learning_rate = 0.001
batch_sizes = [32, 64, 128, 256] 

In [34]:
results = {}
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
# Iterate over different batch sizes
for batch_size in batch_sizes:
    print(f"Evaluating with batch size: {batch_size}")
    
    # Define the model
    model = FFNN(input_size_count, 256, output_size_count, num_layers=2, activation=nn.ReLU())
    
    # Define the loss criterion and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for batch_size in batch_sizes: 
        train_loader_count = DataLoader(train_dataset_count, batch_size, shuffle=True)
        test_loader_count = DataLoader(test_dataset_count, batch_size, shuffle=False)
    
    # Train the model
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader_count:
            optimizer.zero_grad()
            inputs = inputs.float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    accuracy = evaluate_model(model, test_loader_count)
    results[batch_size] = accuracy
    print(f"Accuracy with batch size {batch_size}: {accuracy}")

# Print the results
print("Evaluation results:")
for batch_size, accuracy in results.items():
    print(f"Batch size: {batch_size}, Accuracy: {accuracy}")

Evaluating with batch size: 32


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4551, Precision: 0.3429175033707052, Recall: 0.4532065418237229, F1-Score: 0.36883909788361
Accuracy with batch size 256: None
Evaluating with batch size: 64


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.54675, Precision: 0.46408766866142354, Recall: 0.5481515266229686, F1-Score: 0.49083162620246945
Accuracy with batch size 256: None
Evaluating with batch size: 128


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.73005, Precision: 0.6856766014736746, Recall: 0.7301302971499337, F1-Score: 0.7016503641024141
Accuracy with batch size 256: None
Evaluating with batch size: 256
Accuracy: 0.72445, Precision: 0.6800174701499333, Recall: 0.7235919316364209, F1-Score: 0.6952761456435674
Accuracy with batch size 256: None
Evaluation results:
Batch size: 256, Accuracy: None


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
